# EDA for assesment data

In [1]:
import json
import pandas as pd
import numpy as np
from pprint import pformat
import matplotlib.pyplot as plt


### Load JSON data

In [2]:
with open('../data/assessment_data.json', 'r') as file:
    data = json.load(file)
df = pd.DataFrame(data)

print(df.head())

         Country DeviceCategory EngagementRate  \
0          India        desktop       0.545455   
1  United States        desktop       0.032258   
2  United States        desktop       0.689655   
3  United States        desktop       0.720000   
4  United States        desktop       0.615385   

                               LandingPage  NewUsers  ScreenPageViews  \
0                /blog/what-is-kra-and-kpi        39               46   
1                                (not set)         0                0   
2  /blog/google-sheets-dashboard-templates        24               37   
3        /blog/excel-interactive-dashboard        21               30   
4             /blog/kpi-dashboard-in-excel        11               15   

  SessionMedium  Sessions  TotalUsers  UserEngagementDuration  \
0       organic        44          40                    2526   
1       organic        31          30                     285   
2       organic        29          27                    1072   


### Preprocessing the data

In [3]:
df['date'] = pd.to_datetime(df['date'], format='%a, %d %b %Y %H:%M:%S GMT')

# from string to appropriate numeric types
numeric_columns = ['EngagementRate', 'NewUsers', 'ScreenPageViews', 'Sessions', 'TotalUsers', 'UserEngagementDuration']
remaining_columns = ['Country', 'DeviceCategory', 'LandingPage', 'SessionMedium', 'date']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

### EDA

In [4]:
df.shape

(39850, 11)

In [5]:
df.describe()

,EngagementRate,NewUsers,ScreenPageViews,Sessions,TotalUsers,UserEngagementDuration,date
count,39850.000000,39850.000000,39850.000000,39850.000000,39850.000000,39850.000000,39850
mean,0.487218,1.443463,5.040502,1.948507,2.844918,81.836161,2024-06-15 19:08:44.868256
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2024-05-01 00:00:00
25%,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,2024-05-22 00:00:00
50%,0.500000,1.000000,1.000000,1.000000,1.000000,9.000000,2024-06-16 00:00:00
75%,1.000000,1.000000,2.000000,2.000000,1.000000,71.000000,2024-07-10 00:00:00
max,1.000000,272.000000,2342.000000,273.000000,522.000000,8283.000000,2024-07-31 00:00:00
std,0.468128,3.127523,66.340036,3.642939,20.749115,259.995549,NaN


In [6]:
df.describe(exclude=np.number)

,Country,DeviceCategory,LandingPage,SessionMedium,date
count,39850,39850,39850,39850,39850
unique,198,4,91,6,NaN
top,United States,desktop,/blog/google-sheets-dashboard-templates,organic,NaN
freq,5198,29557,3677,30276,NaN
mean,NaN,NaN,NaN,NaN,2024-06-15 19:08:44.868256
min,NaN,NaN,NaN,NaN,2024-05-01 00:00:00
25%,NaN,NaN,NaN,NaN,2024-05-22 00:00:00
50%,NaN,NaN,NaN,NaN,2024-06-16 00:00:00
75%,NaN,NaN,NaN,NaN,2024-07-10 00:00:00
max,NaN,NaN,NaN,NaN,2024-07-31 00:00:00


In [8]:
print("Missing value counts:")
print(df.isnull().sum())

Missing value counts:
Country                   0
DeviceCategory            0
EngagementRate            0
LandingPage               0
NewUsers                  0
ScreenPageViews           0
SessionMedium             0
Sessions                  0
TotalUsers                0
UserEngagementDuration    0
date                      0
dtype: int64


In [9]:
df['date'].value_counts()

date
2024-07-30    597
2024-05-15    594
2024-07-18    571
2024-07-24    570
2024-05-14    564
             ... 
2024-06-23    244
2024-06-30    238
2024-06-22    219
2024-06-15    216
2024-06-16    194
Name: count, Length: 92, dtype: int64

In [10]:
print(df['date'].min())
print(df['date'].max())

2024-05-01 00:00:00
2024-07-31 00:00:00


In [11]:
df.nunique(axis=0)

Country                    198
DeviceCategory               4
EngagementRate             188
LandingPage                 91
NewUsers                    61
ScreenPageViews            163
SessionMedium                6
Sessions                    66
TotalUsers                 135
UserEngagementDuration    1207
date                        92
dtype: int64

In [12]:
print(sorted(df['DeviceCategory'].unique()))
print(sorted(df['SessionMedium'].unique()))

['desktop', 'mobile', 'smart tv', 'tablet']
['(none)', '(not set)', 'cpc', 'email', 'organic', 'referral']


In [13]:
device_category_counts = df['DeviceCategory'].value_counts().to_dict()
sorted_device_category_counts = dict(sorted(device_category_counts.items()))
print(sorted_device_category_counts)
session_medium_counts = df['SessionMedium'].value_counts().to_dict()
sorted_session_medium_counts = dict(sorted(session_medium_counts.items()))
print(sorted_session_medium_counts)

{'desktop': 29557, 'mobile': 9994, 'smart tv': 4, 'tablet': 295}
{'(none)': 7550, '(not set)': 181, 'cpc': 2, 'email': 8, 'organic': 30276, 'referral': 1833}


In [14]:
country_base_data = df.groupby('Country')['Sessions'].sum().reset_index()
device_base_data = df.groupby('DeviceCategory')['Sessions'].sum().reset_index()


In [15]:
device_base_data

,DeviceCategory,Sessions
0,desktop,60112
1,mobile,17217
2,smart tv,5
3,tablet,314


In [16]:
country_counts = df['Country'].value_counts().to_dict()
country_counts

{'United States': 5198,
 'India': 3669,
 'United Kingdom': 2449,
 'Canada': 1799,
 'Australia': 1720,
 'Indonesia': 1466,
 'Philippines': 1312,
 'Germany': 1021,
 'Malaysia': 953,
 'South Africa': 764,
 'Netherlands': 751,
 'Singapore': 702,
 'Saudi Arabia': 694,
 'France': 687,
 'Pakistan': 667,
 'United Arab Emirates': 667,
 'Ireland': 651,
 'New Zealand': 651,
 'Brazil': 646,
 'Thailand': 580,
 'Egypt': 547,
 'Türkiye': 537,
 'Spain': 529,
 'Vietnam': 492,
 'Nigeria': 479,
 'Mexico': 441,
 'Italy': 434,
 'Kenya': 320,
 'Poland': 296,
 'Bangladesh': 279,
 'Hong Kong': 240,
 'Russia': 239,
 'Portugal': 237,
 'Belgium': 234,
 'Japan': 228,
 'Sri Lanka': 221,
 'Sweden': 221,
 'Israel': 221,
 'Switzerland': 202,
 'Argentina': 197,
 'Ukraine': 193,
 'South Korea': 187,
 'Denmark': 179,
 'Colombia': 179,
 'Qatar': 172,
 'Austria': 169,
 'China': 163,
 'Romania': 160,
 'Taiwan': 145,
 'Morocco': 137,
 'Finland': 120,
 'Chile': 120,
 'Hungary': 119,
 'Oman': 115,
 'Peru': 115,
 'Greece': 115

In [17]:
print(pformat(sorted(df['LandingPage'].unique())))

['(not set)',
 '/',
 '/about-us',
 '/blog',
 '/blog/agencyanalytics-alternatives',
 '/blog/analyze-data',
 '/blog/b2b-marketing-kpis',
 '/blog/best-marketing-dashboard-software',
 '/blog/customer-experience-kpis',
 '/blog/customer-service-kpis',
 '/blog/cyfe-alternatives',
 '/blog/cyfe-vs-geckoboard-vs-datapad',
 '/blog/dashboard-design',
 '/blog/dashboard-reporting-tools',
 '/blog/dashboard-software-for-small-business',
 '/blog/dashboard-tools',
 '/blog/dashthis-alternatives',
 '/blog/databox-alternatives',
 '/blog/databox-pricing',
 '/blog/databox-vs-klipfolio-vs-datapad',
 '/blog/dynamic-dashboard-in-google-sheets',
 '/blog/e-commerce-analytics-tools',
 '/blog/ecommerce-kpi-dashboard',
 '/blog/ecommerce-reporting-tool',
 '/blog/email-marketing-kpis',
 '/blog/excel-dashboard',
 '/blog/excel-interactive-dashboard',
 '/blog/executive-dashboard-software',
 '/blog/executive-reporting',
 '/blog/facebook-kpis',
 '/blog/financial-kpis',
 '/blog/free-dashboard-tools',
 '/blog/geckoboard-alte

In [18]:
landing_page_counts = df['LandingPage'].value_counts().to_dict()
sorted_landing_page_counts = dict(sorted(landing_page_counts.items()))
sorted_landing_page_counts

{'(not set)': 3648,
 '/': 3421,
 '/about-us': 83,
 '/blog': 139,
 '/blog/agencyanalytics-alternatives': 160,
 '/blog/analyze-data': 251,
 '/blog/b2b-marketing-kpis': 17,
 '/blog/best-marketing-dashboard-software': 98,
 '/blog/customer-experience-kpis': 21,
 '/blog/customer-service-kpis': 11,
 '/blog/cyfe-alternatives': 116,
 '/blog/cyfe-vs-geckoboard-vs-datapad': 62,
 '/blog/dashboard-design': 585,
 '/blog/dashboard-reporting-tools': 184,
 '/blog/dashboard-software-for-small-business': 260,
 '/blog/dashboard-tools': 543,
 '/blog/dashthis-alternatives': 182,
 '/blog/databox-alternatives': 459,
 '/blog/databox-pricing': 22,
 '/blog/databox-vs-klipfolio-vs-datapad': 132,
 '/blog/dynamic-dashboard-in-google-sheets': 778,
 '/blog/e-commerce-analytics-tools': 16,
 '/blog/ecommerce-kpi-dashboard': 140,
 '/blog/ecommerce-reporting-tool': 23,
 '/blog/email-marketing-kpis': 6,
 '/blog/excel-dashboard': 1869,
 '/blog/excel-interactive-dashboard': 2555,
 '/blog/executive-dashboard-software': 154,
